In [77]:
import pandas as pd
import arabic
import pyarabic
import helpers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#########################
from numpy import array, argmax
import numpy as np
##########################

In [2]:
data = pd.read_csv('All_clean_data.csv', index_col=0)

c:\users\taha\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\taha\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def string_with_tashkeel_vectorizer(string, tashkeel=arabic.shortharakat):
  '''
      return: 8*1 vector representation for each letter in string
  '''

  # 0* change string to list of letters 
  '''
      * where tshkeel is not considerd a letter
      > Harakah will no be a single member in list
      > it will be concatinated with its previous letter or not exist
  '''
  # factor shaddah and tanwin
  string = helpers.factor_shadda_tanwin(string)

  string_clean = [] # harakah is concatinated with previous latter.
  i = 0
  while True:
      # it is the last item?!
      if i == len(string) - 1:
          string_clean.append(string[i])
          break
      elif i > len(string)-1:
          break
      elif string[i+1] not in tashkeel:
          string_clean.append(string[i])
          i += 1
      elif string[i+1] in tashkeel:
          string_clean.append(string[i] + string[i+1])
          i += 2


  # 1* Building letter and taskell compinations
  arabic_alphabet_tashkeel = helpers.lettersTashkeelCombination

  encoding_combination = array(helpers.encoding_combination)

  # 4* getting encoding for each letter from input string
  representation = []
  for x in string_clean:
      index = string_clean.index(x)
      # Shift index by one 
      representation.append(encoding_combination[index+1])
  
  
  reminder = 111 - len(representation)
  for i in range(reminder):
        representation.append([0,0,0,0,0,0,0,0])
  return np.asarray(representation)


In [4]:
len(string_with_tashkeel_vectorizer('الحمد لله رب العالمين'))

111

In [5]:
data.columns

Index(['العصر', 'الشاعر', 'الديوان', 'القافية', 'البحر', 'الشطر الايسر',
       'الشطر الايمن', 'البيت'],
      dtype='object')

bahr = 'البحر'
bayt = 'البيت'
data  = data[[bayt, bahr]]
data

# Getting Unique Bohor

In [6]:
classic_bohor = data.البحر.unique()[0:16]
classic_bohor

array(['الطويل', 'المنسرح', 'المتقارب', 'الخفيف', 'الكامل', 'السريع',
       'الوافر', 'الرجز', 'البسيط', 'الرمل', 'المجتث', 'المديد', 'الهزج',
       'المتدارك', 'المقتضب', 'المضارع'], dtype=object)

# Filtering on the 16 Bahr

In [7]:
is_classic_bahr = data.البحر.isin(classic_bohor)
filtered_data = data[is_classic_bahr]

# TESTING
print(filtered_data.البحر.unique())
print(len(filtered_data.البحر.unique()))

['الطويل' 'المنسرح' 'المتقارب' 'الخفيف' 'الكامل' 'السريع' 'الوافر' 'الرجز'
 'البسيط' 'الرمل' 'المجتث' 'المديد' 'الهزج' 'المتدارك' 'المقتضب' 'المضارع']
16


In [8]:
maximum_bayt_len = np.max((filtered_data['البيت'].apply(len)))
maximum_bayt_len

111

In [9]:
import h5py

def save(nameOfFile,nameOfDataset,dataVar):
    h5f = h5py.File(nameOfFile, 'w')
    h5f.create_dataset(nameOfDataset, data=dataVar)
    h5f.close()


# Encoding Bohor

In [10]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(classic_bohor)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
bohor_encoded = onehot_encoder.fit_transform(integer_encoded)

In [11]:
print(bohor_encoded.shape)
print(bohor_encoded)
# save('Y_All_Meters', 'Y', bohor_encoded)
type(bohor_encoded)

(16, 16)
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.

numpy.ndarray

# Encoding Abyat

In [12]:
# filtered_data.columns[]
byot = filtered_data['البيت']
bayt_encoded_with_tashkeel = byot.apply(string_with_tashkeel_vectorizer)
#len(bayt_encoded_with_tashkeel[1])


In [61]:
# Reshaping the heck!!
#reshaped_data_matrix_with_tashkeel = np.stack(bayt_encoded_with_tashkeel, axis=0)

# Saving Abyat

In [62]:
# Dimensions
reshaped_data_matrix_with_tashkeel.shape

(1722321, 111, 8)

In [65]:
save('dataMatrix_withTahkeel', 'X', reshaped_data_matrix_with_tashkeel)

In [69]:
bayt_encoded_with_tashkeel.shape

(1722321,)

In [72]:
X = np.array(bayt_encoded_with_tashkeel)
X[1].shape

(111, 8)

In [ ]:
nono = bayt_encoded_with_tashkeel.as_matrix()
np.stack(nono)

In [17]:
test = bayt_encoded_with_tashkeel

In [44]:
test = test[0:2]
test

0    [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
1    [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
Name: البيت, dtype: object

In [59]:
resualt = np.stack(test, axis=0)
print(resualt.shape)
print(resualt)

(2, 111, 8)
[[[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 1 0]
  [0 0 0 ..., 0 1 1]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 1 0]
  [0 0 0 ..., 0 1 1]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]


In [70]:
byot_without_tashkeel = byot.apply(pyarabic.araby.strip_tashkeel)

In [72]:
bayt_encoded_without_tashkeel = byot_without_tashkeel.apply(string_with_tashkeel_vectorizer)

In [73]:
bayt_encoded_without_tashkeel

0          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
1          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
2          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
3          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
4          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
5          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
6          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
7          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
8          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
9          [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
10         [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
11         [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
12         [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
13         [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
14         [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
15         [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...
16         [[0, 0, 0, 0,

In [76]:
reshaped_data_matrix_without_tashkeel = np.stack(bayt_encoded_without_tashkeel, axis=0)

MemoryError: 